In [ ]:
import os
import csv
import numpy as np
import pandas as pd
import os
import re
import pickle
import itertools
import collections
import tqdm
import random
import sys

In [ ]:
data_file_path = "Bases/TestePerplexidade/arqPerplexidade.csv"
data_file_exists = os.path.isfile(data_file_path)
data_file = open(data_file_path, "a", newline="")
csv_writer = csv.writer(data_file, delimiter=";")

if not data_file_exists:
    csv_writer.writerow(["id", "frase", "unigrama", "bigrama", "trigrama", "interpolação"])

In [ ]:
def save_data(
    data_id,
    frase,
    unigrama,
    bigrama,
    trigrama,
    interpolação,
):

    csv_writer.writerow([data_id, frase, unigrama, bigrama, trigrama, interpolação])

    return data_id

In [ ]:
# Ler Arquivos de Teste
arquivosTest = list(os.walk('Bases/TestePerplexidade/Test/'))[0][2]

corpusTest = ''
for a in arquivosTest:
        with open('Bases/TestePerplexidade/Test/' + a,'r', encoding='iso-8859-1') as f:
                corpusTest = corpusTest + f.read()

In [ ]:
contador_arquivo={}
with open('Bases/TestePerplexidade/recPerplexidade.pickle', 'rb') as handle:
    contador_arquivo = pickle.load(handle)

In [ ]:
def tokenize (txt):
    p = re.finditer ('(?:<TEXT>).*?(?:</TEXT>)',txt,flags=re.DOTALL|re.MULTILINE)
    for a in p:
        for x in re.finditer('[a-zA-ZâêîôûáéíóúàèìòùãẽĩõũçÁÉÍÓÚÀÈÌÒÙÃẼĨÕŨÂÊÎÔÛÇ]+|<TEXT>|</TEXT>|\.',a.group(),flags=re.DOTALL|re.MULTILINE):
            t = x.group()
            if t=='<TEXT>':
                yield '<s>'
                yield '<s>'
                yield '<s>'
            elif t == '</TEXT>':
                yield '</s>'
                yield '</s>'
                yield '</s>'
            elif t == '.':
                yield '</s>'
                yield '</s>'
                yield '</s>'
                yield '<s>'
                yield '<s>'
                yield '<s>'
            else:
                yield t.lower()

In [ ]:
def slidingWindow(winsize):
    def itt(tokenizedText):
        iters=itertools.tee(tokenizedText,winsize)
        for x in range(winsize):
            for i in range(x):
                next(iters[x])
        while True:
            try:
                yield tuple([next(it) for it in iters])
            except StopIteration:
                return
    return itt

In [ ]:
f4 = slidingWindow(4)
f3 = slidingWindow(3)
f2 = slidingWindow(2)
total = 21016255

In [ ]:
def analisaFrase(x, w1, w2, w3):
    xx = tokenize(x)
    quatro = f4(tokenize(x))
    tres = f3(tokenize(x))
    dois = f2(tokenize(x))
    next(dois)
    um = tokenize(x)
    next(um)
    next(um)
    prod=0
    i=0
    smoothing_factor=0.1
    epsilon =1e-10
    for w in zip(um, dois,tres):
        #p3 = epsilon
        #p2 = epsilon 
        #if contador_arquivo[w[2][0]][w[2][1]][None] > 0: 
        if w[2][0] in contador_arquivo and w[2][1] in contador_arquivo[w[2][0]] and w[2][2] in contador_arquivo[w[2][0]][w[2][1]]:
            p3 = (contador_arquivo[w[2][0]][w[2][1]][w[2][2]][None]) / (contador_arquivo[w[2][0]][w[2][1]][None])
            
        #if contador_arquivo[w[1][0]][None] > 0:
        if w[1][0] in contador_arquivo and w[1][1] in contador_arquivo[w[1][0]]:
            p2 = (contador_arquivo[w[1][0]][w[1][1]][None]) / (contador_arquivo[w[1][0]][None])
        
        if w[0] in contador_arquivo:   
            p1 = (contador_arquivo[w[0]][None]) / (total)
        
        prod = prod + np.log(w1*p1 + w2*p2 + w3*p3)
        i+=1
    return 1/((np.exp(prod))**(1/i))


In [ ]:
# Example usage
analisaFrase('<TEXT></TEXT>', 1, 0, 0)

In [ ]:
# Expressão regular para extrair o texto entre <TEXT></TEXT>
padrao_texto = r'<TEXT>(.*?)<\/TEXT>'

# Encontrar todas as ocorrências de texto entre as tags <TEXT></TEXT>
ocorrencias_texto = re.findall(padrao_texto, corpusTest, re.DOTALL)

# Inicializar lista para armazenar frases de todas as ocorrências
frases = []

# Iterar sobre as ocorrências de texto e extrair as frases
for ocorrencia in ocorrencias_texto:
    # Expressão regular para extrair frases completas excluindo palavras entre <> e que terminam com ponto, ponto de interrogação ou ponto de exclamação
    padrao_frases = r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|!)\s'
    # Extrair as frases completas
    frases_ocorrencia = re.split(padrao_frases, ocorrencia)
    # Adicionar as frases desta ocorrência à lista de frases
    frases.extend(frases_ocorrencia)

# Remover frases vazias e múltiplos espaços consecutivos
frases = [' '.join(re.findall(r'\S+', frase)) for frase in frases if frase.strip()]

# Remover ponto e vírgula (;) das frases
data_id = 0
for frase in frases:
    data_id += 1
    frases = [frase.replace(';', '')]
    # Unigrama
    try:
        unigrama = analisaFrase('<TEXT>'+frase+'</TEXT>', 1,0,0)
    except Exception as e:
        unigrama = "" 

    # Bigrama
    try:
        bigrama = analisaFrase('<TEXT>'+frase+'</TEXT>', 0,1,0)
    except Exception as e:
        bigrama = "" 
        
    # Trigrama
    try:
        trigrama = analisaFrase('<TEXT>'+frase+'</TEXT>', 0,0,1)
    except Exception as e:
        trigrama = "" 

    # Interpolação
    try:
        interpolação = analisaFrase('<TEXT>'+frase+'</TEXT>', 0.1,0.2,0.7)
    except Exception as e:
        interpolação = "" 
    
    try:
        num_images = save_data(
            data_id,
            frase,
            unigrama,
            bigrama,
            trigrama,
            interpolação,
        )
    except Exception as e:
        pass